In [25]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import requests
import json
import re
from multiprocessing.dummy import Pool
import random
import time
import math
from lxml import etree

In [26]:
#搜索页面
init_search = 'https://coll.jd.com/list.html?sub={}&sort=sort_totalsales15_desc&stock=0&page='

#商品页面
init_itemUrl = 'https://item.jd.com/{}.html'

headers={
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }

init_commentUrl = 'https://sclub.jd.com/comment/productPageComments.action?productId={}&score=0&sortType=5&page={}&pageSize=10&isShadowSku=0&fold=1'
init_priceUrl = 'https://p.3.cn/prices/mgets?skuIds=J_{}'

In [27]:
def openlink(link):
    maxTryNum = 10
    for tries in range(0,maxTryNum):
        try:
            res = requests.get(link, headers = headers)
            return res
        except:
            if tries < (maxTryNum - 1):
                print("啊呀")
                continue
            else:
                print("Has tried %d times to access url %s, all failed!", maxTryNum, link)
                break

In [28]:
def getitemUrl(target,maxpage = 1):
    itemList = []
    for i in range(1,maxpage+1):
        url = init_search.format(target)  + str(i)
        res = requests.get(url,headers=headers)
        selector = etree.HTML(res.text)
        itemList.extend(selector.xpath('//div[@id="plist"]/ul/li/div/div[@class="p-name"]/a/@href'))
    return itemList

In [29]:
def getitemidList(target,maxpage = 1):
    itemList = []
    itemidList = []
    for i in range(1,maxpage+1):
        url = init_search.format(target)  + str(i)
        res = requests.get(url,headers=headers)
        selector = etree.HTML(res.text)
        itemList.extend(selector.xpath('//div[@id="plist"]/ul/li/div/div[@class="p-name"]/a/@href'))
        
    print('页面获取完成，共有%d页%d个商品'%(maxpage,len(itemList)))
    i = 1
    for itemUrl in itemList:
        res = requests.get('https:'+itemUrl,headers=headers).text
        selector = etree.HTML(res)
        itemidList.extend(selector.xpath('//div[@id="choose-attrs"]/div//div[@class="dd"]/div/@data-sku')) 
        print('\r' + str(i) + '/' + str(len(itemList)),end = '')
        i += 1
    print('所有分类商品获取完成，共有%d个商品'%(len(itemidList)))
    return itemidList

In [30]:
def getitemInfo(itemid):
    itemUrl = init_itemUrl.format(itemid)
    res = openlink(itemUrl)
    selector = etree.HTML(res.text)

    title = "".join(selector.xpath(
                    '//div[@class="sku-name"]/text()')).replace(' ', '').replace('\r', '').replace('\n', '')

    itemDetail = ";".join(selector.xpath('//div[@class="p-parameter"]/ul[@class="parameter2 p-parameter-list"]/li/text()'))

    content = json.loads(requests.get(init_priceUrl.format(itemid)).text)
    price = content[0]['p']

    comment = json.loads(requests.get(init_commentUrl.format(itemid,0)).text)
    goodRate = comment['productCommentSummary']['goodRate']
    commentCount= comment['productCommentSummary']['commentCountStr']
    itemInfo = pd.DataFrame(dict(itemUrl = [itemUrl],
                                itemid = [itemid],
                                title = [title],
                                itemDetail = [itemDetail],
                                price = [price],
                                goodRate = [goodRate],
                                commentCount = [commentCount]),
                                columns=['itemid',
                                            'itemUrl',
                                            'title',
                                            'itemDetail',
                                            'price' ,
                                            'goodRate',
                                            'commentCount'])
    print('#',end = '')
    return itemInfo
    

In [42]:
def getitemComment(itemid):
    itemUrl = init_itemUrl.format(itemid)
    res = openlink(init_commentUrl.format(itemid,0))
    jc = json.loads(res.text)
    maxPage = jc['maxPage']
    info = []
    pagelist = list(range(0,maxPage+1))

    getitemComment_eachpage(itemid,pagelist,info)
    itemcomments = pd.DataFrame
    #   把info这个list的每个dataframe拼成一个dataframe\
    print("该商品所有页面都爬完了")
    if len(info) != 0:
        itemcomments = pd.concat(info,axis = 0 ,ignore_index=True)
        #itemcomments.to_csv('comments.csv',mode = 'a+',header=False,index=False)
        itemcomments.to_csv('output/'+'行车记录仪'+'-comments-'+str(time.strftime("%Y-%m-%d"))+'.csv',mode = 'a+',header=False,index=False)
    else :
        print("没评论")
    return  itemcomments 

In [45]:
def getitemComment_eachpage(itemid,pagelist,info):
    itemUrl = init_itemUrl.format(itemid)
    banlist = []
    for page in pagelist:
        res = openlink(init_commentUrl.format(itemid,page))
        if len(res.text):
            jc = json.loads(res.text)
            if 'comments' in jc:
                commentlist = jc['comments']
                for comment in commentlist:
                    nickname = comment['nickname']
                    if 'productColor' in comment:
                        productColor  = comment['productColor']
                    else :
                        productColor = None
                    score = comment['score']
                    content = comment['content']
                    creationTime = comment['creationTime']
                    if 'afterUserComment' in comment:
                        afterUserComment = comment['afterUserComment']['hAfterUserComment']['content']
                        afterDays = comment['afterUserComment']['created']
                    else :
                        afterUserComment= None
                        afterDays= None
                    info.append(pd.DataFrame(dict(itemid = [itemid],
                                                          itemUrl = [itemUrl],
                                                          nickname = [nickname],
                                                          productColor = [productColor],
                                                          score = [score],
                                                          content = [content],
                                                          creationTime = [creationTime],
                                                          afterUserComment = [afterUserComment],
                                                          afterDays = [afterDays]),columns = ['itemid' , 'itemUrl' , 'nickname','productColor','score','content','creationTime','afterUserComment','afterDays']))
                    print('\r    第%2d/%2d页 '%(page,len(pagelist)-1),end='')
            else :
                print('被干掉了等一等')
                banlist.append(page)
                time.sleep(random.uniform(1,6))
    if len(banlist) != 0:
        info.append(getitemComment_eachpage(itemid,banlist,info))
    else:
        return info

## 测试代码

In [46]:
target = 23931
itemidList = getitemidList(target,1)

页面获取完成，共有1页60个商品
60/60所有分类商品获取完成，共有215个商品


### 并行爬取商品信息

In [31]:
pool = Pool(8)
time1 =time.time()
itemInfo = pool.map(getitemInfo,itemidList)
pool.close()
pool.join()
time2 =time.time()
print(str(time2-time1))

#######################################################################################################################################################################################################################24.151516914367676


In [32]:
info = pd.concat(itemInfo,ignore_index=True)

In [33]:
info.to_csv('jd行车记录仪信息'+str(time.strftime("%Y-%m-%d"))+'.csv',encoding='utf-8')

### 并行爬取商品评论

In [48]:
pool = Pool(8)
time1 =time.time()
itemInfo = pool.map(getitemComment,itemidList)
pool.close()
pool.join()
time2 =time.time()
print(str(time2-time1))

    第38/100页 该商品所有页面都爬完了
    第66/100页 该商品所有页面都爬完了
    第92/100页 该商品所有页面都爬完了
    第25/61页 该商品所有页面都爬完了
    第97/100页 该商品所有页面都爬完了
    第98/100页 该商品所有页面都爬完了
    第 7/100页 该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了
    第41/100页 该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了
    第97/100页 该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了
    第78/100页 该商品所有页面都爬完了
    第91/100页 该商品所有页面都爬完了
    第93/100页 该商品所有页面都爬完了
    第13/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
    第 1/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
    第 1/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
该商品所有页面都爬完了
    第 0/100页 该商品所有页面都爬完了
    第 1/100页 该商品所有页面都爬完了
    第 2/61页  该商品所有页面都爬完了
    第 2/47页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
    第 0/100页 该商品所有页面都爬完了
    第 2/100页 该商品所有页面都爬完了
该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了
    第32/61页 该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了
    第45/61页 该商品所有页面都爬完了
    第89/100页 该商品所有页面都爬完了
    第100/100页 该商品所有页面都爬完了


In [61]:
df = pd.read_csv('output/123.csv')

In [63]:
df.columns = ['itemid' , 'itemUrl' , 'nickname','productColor','score','content','creationTime','afterUserComment','afterDays']

In [64]:
df

,itemid,itemUrl,nickname,productColor,score,content,creationTime,afterUserComment,afterDays
0,8035688,https://item.jd.com/8035688.html,w***m,【新品】小米 后视镜记录仪,5.0,性价比比较好，希望质量也不错，安装后我在追评吧。我选的只是后视镜，没有选储存卡，所以大家购买...,2018-08-06 20:54:45,NaN,NaN
1,8035688,https://item.jd.com/8035688.html,8***g,【7月6日首发】后视镜记录仪,5.0,还不错 简单 方便 大方 明了 避免不必要的麻烦 不错的选择,2018-07-19 04:10:18,NaN,NaN
2,8035688,https://item.jd.com/8035688.html,a***d,【7月6日首发】后视镜记录仪,5.0,行车记录仪是挺好用的，就是自己的车用不了，倒车影像后摄像头。,2018-07-26 18:13:29,NaN,NaN
3,8035688,https://item.jd.com/8035688.html,j***g,【新品】小米 后视镜记录仪,5.0,质量不错，画面也很清晰，可以连接上手机，很方便,2018-08-06 22:40:43,NaN,NaN
4,8035688,https://item.jd.com/8035688.html,广***田,【7月6日首发】后视镜记录仪,5.0,不错的一款记录仪！比我以前买的不论是外观还是防抖性！还可以语音！,2018-08-05 11:23:19,NaN,NaN
5,8035688,https://item.jd.com/8035688.html,園來如此,【7月6日首发】后视镜记录仪,5.0,挺不错，一直用小米的，这次又买了一个，好评，,2018-07-09 14:14:54,NaN,NaN
6,8035688,https://item.jd.com/8035688.html,j***8,【新品】小米 后视镜记录仪,5.0,安上去很方便，功能强大。谢谢小米的产品,2018-08-11 20:33:35,NaN,NaN
7,8035688,https://item.jd.com/8035688.html,H***c,【7月6日首发】后视镜记录仪,5.0,一直在等小米，一出来果断下单，京东自营值得信赖，赞一个,2018-08-06 13:21:11,NaN,NaN
8,8035688,https://item.jd.com/8035688.html,jd135469aut,【7月6日首发】后视镜记录仪,5.0,目前看着还不错。。。。。。。。。。,2018-08-03 00:13:35,NaN,NaN
9,8035688,https://item.jd.com/8035688.html,冀***东,【新品】小米 后视镜记录仪,5.0,东西还是可以的，但是感觉充电器塑料的质量很差,2018-08-12 16:56:14,NaN,NaN
